# SQL Homework
Author: Hao CHEN
</br>
</br>
**Note**: 
- It takes 2-3 minutes to run this notebook, because it contains a program that automatically downloads all data from https://swapi.dev. 
- This notebook is very long, so I have created headings for you to collapse. Please use it to ease the reading process.

### Import Libraries

In [1]:
import pandas as pd
import numpy as np
import sqlite3
import json
import requests

## 1. Data Engineering - ETL

### 1.1 Extract

#### 1.1.1 Download Datasets

In [2]:
# Define a function to extract the online dictionary of urls for all datasets
def extract_resource_dict():
    r = requests.get('https://swapi.dev/api/')
    resource_dict = r.json()
    return resource_dict

# Test the function
resource_dict = extract_resource_dict()
print(resource_dict)

{'people': 'https://swapi.dev/api/people/', 'planets': 'https://swapi.dev/api/planets/', 'films': 'https://swapi.dev/api/films/', 'species': 'https://swapi.dev/api/species/', 'vehicles': 'https://swapi.dev/api/vehicles/', 'starships': 'https://swapi.dev/api/starships/'}


In [3]:
# Define a function to automatically download dataset 
# The function also automatically turns the dataset into dataframe
def extract_resource(resource):
    # Extract the resource dictionary
    resource_dict = extract_resource_dict()
    
    # Initialise search url
    search_url = resource_dict[resource]

    # Initialise results list
    results = []

    # Start searching
    print(f"Start Extracting Dataset: {resource}")

    # Looping over search_urls for the source 
    while search_url is not None:
        print(f'Searched URL: {search_url}')
        request = requests.get(search_url)
        file = request.json()
        results += file['results']
        # Set the search_url to next search 
        search_url = file['next']
    
    # Return a dataframe of the result
    print('Finish Extraction. Returning the data as a pd.Dataframe')
    return pd.DataFrame(results)

In [4]:
# Extract data
films = extract_resource('films')
people = extract_resource('people')
planets = extract_resource('planets')
species = extract_resource('species')
starships = extract_resource('starships')
vehicles = extract_resource('vehicles')

Start Extracting Dataset: films
Searched URL: https://swapi.dev/api/films/
Finish Extraction. Returning the data as a pd.Dataframe
Start Extracting Dataset: people
Searched URL: https://swapi.dev/api/people/
Searched URL: https://swapi.dev/api/people/?page=2
Searched URL: https://swapi.dev/api/people/?page=3
Searched URL: https://swapi.dev/api/people/?page=4
Searched URL: https://swapi.dev/api/people/?page=5
Searched URL: https://swapi.dev/api/people/?page=6
Searched URL: https://swapi.dev/api/people/?page=7
Searched URL: https://swapi.dev/api/people/?page=8
Searched URL: https://swapi.dev/api/people/?page=9
Finish Extraction. Returning the data as a pd.Dataframe
Start Extracting Dataset: planets
Searched URL: https://swapi.dev/api/planets/
Searched URL: https://swapi.dev/api/planets/?page=2
Searched URL: https://swapi.dev/api/planets/?page=3
Searched URL: https://swapi.dev/api/planets/?page=4
Searched URL: https://swapi.dev/api/planets/?page=5
Searched URL: https://swapi.dev/api/plane

#### 1.1.2 Quickly Check Imported Datasets

In [5]:
# Define a function to quickly inspect the properties of dataframes
def quick_inspect(dataframe):
    print(f'Rows: {dataframe.shape[0]}; Columns: {dataframe.shape[1]}')
    return dataframe.head(1)

In [6]:
quick_inspect(films)

Rows: 6; Columns: 14


,title,episode_id,opening_crawl,director,producer,release_date,characters,planets,starships,vehicles,species,created,edited,url
0,A New Hope,4,It is a period of civil war.\r\nRebel spaceshi...,George Lucas,"Gary Kurtz, Rick McCallum",1977-05-25,"[https://swapi.dev/api/people/1/, https://swap...","[https://swapi.dev/api/planets/1/, https://swa...","[https://swapi.dev/api/starships/2/, https://s...","[https://swapi.dev/api/vehicles/4/, https://sw...","[https://swapi.dev/api/species/1/, https://swa...",2014-12-10T14:23:31.880000Z,2014-12-20T19:49:45.256000Z,https://swapi.dev/api/films/1/


In [7]:
quick_inspect(people)

Rows: 82; Columns: 16


,name,height,mass,hair_color,skin_color,eye_color,birth_year,gender,homeworld,films,species,vehicles,starships,created,edited,url
0,Luke Skywalker,172,77,blond,fair,blue,19BBY,male,https://swapi.dev/api/planets/1/,"[https://swapi.dev/api/films/1/, https://swapi...",[],"[https://swapi.dev/api/vehicles/14/, https://s...","[https://swapi.dev/api/starships/12/, https://...",2014-12-09T13:50:51.644000Z,2014-12-20T21:17:56.891000Z,https://swapi.dev/api/people/1/


In [8]:
quick_inspect(planets)

Rows: 60; Columns: 14


,name,rotation_period,orbital_period,diameter,climate,gravity,terrain,surface_water,population,residents,films,created,edited,url
0,Tatooine,23,304,10465,arid,1 standard,desert,1,200000,"[https://swapi.dev/api/people/1/, https://swap...","[https://swapi.dev/api/films/1/, https://swapi...",2014-12-09T13:50:49.641000Z,2014-12-20T20:58:18.411000Z,https://swapi.dev/api/planets/1/


In [9]:
quick_inspect(species)

Rows: 37; Columns: 15


,name,classification,designation,average_height,skin_colors,hair_colors,eye_colors,average_lifespan,homeworld,language,people,films,created,edited,url
0,Human,mammal,sentient,180,"caucasian, black, asian, hispanic","blonde, brown, black, red","brown, blue, green, hazel, grey, amber",120,https://swapi.dev/api/planets/9/,Galactic Basic,"[https://swapi.dev/api/people/66/, https://swa...","[https://swapi.dev/api/films/1/, https://swapi...",2014-12-10T13:52:11.567000Z,2014-12-20T21:36:42.136000Z,https://swapi.dev/api/species/1/


In [10]:
quick_inspect(starships)

Rows: 36; Columns: 18


,name,model,manufacturer,cost_in_credits,length,max_atmosphering_speed,crew,passengers,cargo_capacity,consumables,hyperdrive_rating,MGLT,starship_class,pilots,films,created,edited,url
0,CR90 corvette,CR90 corvette,Corellian Engineering Corporation,3500000,150,950,30-165,600,3000000,1 year,2.0,60,corvette,[],"[https://swapi.dev/api/films/1/, https://swapi...",2014-12-10T14:20:33.369000Z,2014-12-20T21:23:49.867000Z,https://swapi.dev/api/starships/2/


In [11]:
quick_inspect(vehicles)

Rows: 39; Columns: 16


,name,model,manufacturer,cost_in_credits,length,max_atmosphering_speed,crew,passengers,cargo_capacity,consumables,vehicle_class,pilots,films,created,edited,url
0,Sand Crawler,Digger Crawler,Corellia Mining Corporation,150000,36.8,30,46,30,50000,2 months,wheeled,[],"[https://swapi.dev/api/films/1/, https://swapi...",2014-12-10T15:36:25.724000Z,2014-12-20T21:30:21.661000Z,https://swapi.dev/api/vehicles/4/


### 1.1.3 Raw Data Structure

I have created a google excel document listing each raw table with their variables. Please see the link below:

https://docs.google.com/spreadsheets/d/162WwowSMNLXuTcYaetTW22INjrI6bQQOx-cRzQgTers/edit?usp=sharing 

Here is a screenshot of the excel:

<img src="Pictures/raw_tables.png" width="500" />


### 1.2 Transform

In [12]:
# Store all dataframe into one dictionary
dataframes = {
    'films': films, 
    'people': people,
    'planets': planets,
    'species': species,
    'starships': starships,
    'vehicles': vehicles
}

# Initialised a dictionary to store cleaned dataframes
dataframes_cleaned = {}

#### 1.2.1 Convert URls into Primary Keys for All Tables

Each raw table contains a url column, which can be transformed into primary key. See an example below.

<img src="Pictures/create_keys.png" width="450" />


In [13]:
# generate primary keys
for name, df in dataframes.items():
    df['id'] = df['url'].str.extract(r'(\d+)/$').astype('Int64')

# Double check
films[['id', 'url']].head(2)

,id,url
0,1,https://swapi.dev/api/films/1/
1,2,https://swapi.dev/api/films/2/


#### 1.2.2 Convert date strings into timestamps

In [14]:
# create timestamps for the created and edited time of each entry
for name, df in dataframes.items():
    df['created'] = pd.to_datetime(df['created'], format='ISO8601')
    df['edited'] = pd.to_datetime(df['edited'], format='ISO8601')

# double check
print(films['created'].dtypes)
print(films['edited'].dtypes)

datetime64[ns, UTC]
datetime64[ns, UTC]


In [15]:
# Convert release-date of films into timestamps
films['release_date'] = pd.to_datetime(films['release_date'])

# Double Check
print(films['release_date'].dtypes)

datetime64[ns]


#### 1.2.3.1 Build junction tables for films & other datasets

The 'films' table have many-to-many relationships with all other tables. For example, a character has shown up in multiple films, and a film have multiple characters. See this picture below:

<img src="Pictures/junction_tables_before.png" width="650" />

Hence, junctions table are built below to solve the many-to-many (M:N) relationships between 'films' and other tables.

See the picture below for the relationships between table after the processing:

<img src="Pictures/junction_tables_after.png" width="800" />

In [16]:
# Initialise a dictionary of junction tables
junction_tables = {}


# Defining specifications for junction tables
junction_tables_info = [
    {'name': 'character_in_film', 'expand_col': 'characters', 'new_id': 'character_id'},
    {'name': 'planet_in_film', 'expand_col': 'planets', 'new_id': 'planet_id'},
    {'name': 'starship_in_film', 'expand_col': 'starships', 'new_id': 'starship_id'},
    {'name': 'species_in_film', 'expand_col': 'species', 'new_id': 'species_id'},
    {'name': 'vehicle_in_film', 'expand_col': 'vehicles', 'new_id': 'vehicle_id'}
]

# Iteratively generate junction tables based on the specifications above
# And store generated tables in the junction_tables dictionary
for table in junction_tables_info:

    # Extracting and renaming necessary columns 
    temp_df = films[['id', table['expand_col']]]
    temp_df = temp_df.rename(columns={'id': 'film_id'})

    # Expand rows with multiple entries in the respective column
    temp_df = temp_df.explode(table['expand_col'])

    # Extract relevant IDs and rename columns
    temp_df[table['new_id']] = temp_df[table['expand_col']].str.extract(r'(\d+)/$').astype(int)
    temp_df = temp_df.drop(columns=[table['expand_col']])
    
    # Reset dataframe index
    temp_df = temp_df.reset_index(drop=True, inplace=False)
    
    # Save the generated junction_table into the dictionary 
    junction_tables[table['name']] = temp_df

    # Clean films dataframe by dropping the processed column
    films = films.drop(columns=table['expand_col'])

In [17]:
# Double check junction tables
print(junction_tables['character_in_film'])
print(junction_tables['planet_in_film'])
print(junction_tables['starship_in_film'])
print(junction_tables['species_in_film'])
print(junction_tables['vehicle_in_film'])

     film_id  character_id
0          1             1
1          1             2
2          1             3
3          1             4
4          1             5
..       ...           ...
157        6            79
158        6            80
159        6            81
160        6            82
161        6            83

[162 rows x 2 columns]
    film_id  planet_id
0         1          1
1         1          2
2         1          3
3         2          4
4         2          5
5         2          6
6         2         27
7         3          1
8         3          5
9         3          7
10        3          8
11        3          9
12        4          1
13        4          8
14        4          9
15        5          1
16        5          8
17        5          9
18        5         10
19        5         11
20        6          1
21        6          2
22        6          5
23        6          8
24        6          9
25        6         12
26        6         13
27      

#### 1.2.3.2 Clearn films table

In [18]:
# Select needed rows 
selected_columns = ['id', 'episode_id', 'title', 'release_date', 'created', 'edited', 'url']
films_cleaned = films[selected_columns]

In [19]:
# Store cleaned dataframe
dataframes_cleaned['films'] = films_cleaned

In [20]:
# Final dataset
dataframes_cleaned['films'] 

,id,episode_id,title,release_date,created,edited,url
0,1,4,A New Hope,1977-05-25,2014-12-10 14:23:31.880000+00:00,2014-12-20 19:49:45.256000+00:00,https://swapi.dev/api/films/1/
1,2,5,The Empire Strikes Back,1980-05-17,2014-12-12 11:26:24.656000+00:00,2014-12-15 13:07:53.386000+00:00,https://swapi.dev/api/films/2/
2,3,6,Return of the Jedi,1983-05-25,2014-12-18 10:39:33.255000+00:00,2014-12-20 09:48:37.462000+00:00,https://swapi.dev/api/films/3/
3,4,1,The Phantom Menace,1999-05-19,2014-12-19 16:52:55.740000+00:00,2014-12-20 10:54:07.216000+00:00,https://swapi.dev/api/films/4/
4,5,2,Attack of the Clones,2002-05-16,2014-12-20 10:57:57.886000+00:00,2014-12-20 20:18:48.516000+00:00,https://swapi.dev/api/films/5/
5,6,3,Revenge of the Sith,2005-05-19,2014-12-20 18:49:38.403000+00:00,2014-12-20 20:47:52.073000+00:00,https://swapi.dev/api/films/6/


#### 1.2.4 Clean people table

In [21]:
# Convert urls in 'species' column into species ids
people['species'] = people['species'].apply(
    lambda x: None if len(x) == 0 else x[0])

people['species'] = people['species'].str.extract(r'(\d+)/$').astype('Int64')

In [22]:
# Convert urls in 'homeworld' column into planet ids
# Rename 'homeworld' column to 'residence_planet'
people['homeworld'] = people['homeworld'].str.extract(r'(\d+)/$').astype('Int64')
people = people.rename(columns={'homeworld': 'residence_planet'})

In [23]:
# Convert height & mass columns (string) to int or float

# Replace 'unknown' with NaN
people['height_int'] = people['height'].replace('unknown', pd.NA)
people['mass_float'] = people['mass'].replace('unknown', pd.NA)

# Delete commas within number, e.g., 1,300 -> 1300
people['height_int'] = people['height_int'].str.replace(',', '')
people['mass_float'] = people['mass_float'].str.replace(',', '')

# Convert to int or float
people['height_int'] = people['height_int'].astype('Int64')
people['mass_float'] = pd.to_numeric(people['mass_float'], errors='coerce')

# Rename columns
people['height'] = people['height_int']
people['mass'] = people['mass_float']

# Double check
print(people['height'].dtypes)
print(people['mass'].dtypes)


Int64
float64


In [24]:
# Select needed columns
selected_columns = ['id', 'name', 'height', 'mass', 'birth_year', 
                    'gender', 'residence_planet', 'species',
                    'created', 'edited', 'url']

people_cleaned = people[selected_columns]

# Store cleaned dataframe
dataframes_cleaned['people'] = people_cleaned

In [25]:
# Final cleaned people dataframe
dataframes_cleaned['people'].head(2)

,id,name,height,mass,birth_year,gender,residence_planet,species,created,edited,url
0,1,Luke Skywalker,172,77.0,19BBY,male,1,<NA>,2014-12-09 13:50:51.644000+00:00,2014-12-20 21:17:56.891000+00:00,https://swapi.dev/api/people/1/
1,2,C-3PO,167,75.0,112BBY,n/a,1,2,2014-12-10 15:10:51.357000+00:00,2014-12-20 21:17:50.309000+00:00,https://swapi.dev/api/people/2/


#### 1.2.5 Clearning planets dataframe

In [26]:
# Convert population column (string) to int

# Replace 'unknown' with NaN
planets['population_int'] = planets['population'].replace('unknown', pd.NA)

# Delete commas within number, e.g., 1,300 -> 1300
planets['population_int'] = planets['population_int'].str.replace(',', '')

# Convert to int
planets['population_int'] = planets['population_int'].astype('Int64')

# Rename columns
planets['population'] = planets['population_int']

# Double check
print(planets['population'].dtypes)

Int64


In [27]:
# Convert following columns (string) to int/float:
# rotation_period, orbital_period, diameter, surface_water 

# rotation_period
planets['rotation_period'] = planets['rotation_period'].replace('unknown', pd.NA).astype('Int64')
# planets['rotation_period'] = planets['rotation_period_int']

# orbital_period
planets['orbital_period'] = planets['orbital_period'].replace('unknown', pd.NA).astype('Int64')

# diameter
planets['diameter'] = planets['diameter'].replace('unknown', pd.NA).astype('Int64')

# surface_water
planets['surface_water'] = pd.to_numeric(planets['surface_water'].replace('unknown', pd.NA),
                                               errors='coerce')

# Double check
print(planets['rotation_period'].dtypes)
print(planets['orbital_period'].dtypes)
print(planets['diameter'].dtypes)
print(planets['surface_water'].dtypes)

Int64
Int64
Int64
float64


In [28]:
# Select needed columns for planet

selected_columns = ['id', 'name', 'rotation_period', 'orbital_period',
                    'diameter', 'climate', 'surface_water', 'population',
                    'created', 'edited', 'url']

planets_cleaned = planets[selected_columns]

# Store cleaned dataframe
dataframes_cleaned['planets'] = planets_cleaned

In [29]:
# Final data for planets
dataframes_cleaned['planets'].head(1)

,id,name,rotation_period,orbital_period,diameter,climate,surface_water,population,created,edited,url
0,1,Tatooine,23,304,10465,arid,1.0,200000,2014-12-09 13:50:49.641000+00:00,2014-12-20 20:58:18.411000+00:00,https://swapi.dev/api/planets/1/


#### 1.2.6 Clearning species dataframe

In [30]:
# Create home_planet column as foreign key
species['home_planet'] = species['homeworld'].str.extract(r'(\d+)/$').astype('Int64')

In [31]:
# Convert average_height & average_lifespan (string) into int/float 

# Replace 'unknown' with NaN
replace_string = ['unknown', 'n/a', 'indefinite']
species['average_height_int'] = species['average_height'].replace(replace_string, pd.NA)
species['average_lifespan_int'] = species['average_lifespan'].replace(replace_string, pd.NA)

# Convert to int or float
species['average_height_int'] = species['average_height_int'].astype('Int64')
species['average_lifespan_int'] = species['average_lifespan_int'].astype('Int64')

# Rename columns
species['average_height'] = species['average_height_int']
species['average_lifespan'] = species['average_lifespan_int']

# Double check
print(species['average_height'].dtypes)
print(species['average_lifespan'].dtypes)


Int64
Int64


In [32]:
# Missing values in language: replace 'unknown' & n/a with NaN 
# Replace 'unknown' with NaN
replace_string = ['unknown', 'n/a']
species['language'] = species['language'].replace(replace_string, pd.NA)

In [33]:
# Select needed columns
selected_columns = ['id', 'name', 'classification',
                    'average_height', 'average_lifespan', 
                    'home_planet', 'language',
                    'created', 'edited', 'url']
species_cleaned = species[selected_columns]

# Store dataframes
dataframes_cleaned['species'] = species_cleaned

In [34]:
# Final cleaned data for species
dataframes_cleaned['species'].head(2)

,id,name,classification,average_height,average_lifespan,home_planet,language,created,edited,url
0,1,Human,mammal,180,120,9,Galactic Basic,2014-12-10 13:52:11.567000+00:00,2014-12-20 21:36:42.136000+00:00,https://swapi.dev/api/species/1/
1,2,Droid,artificial,<NA>,<NA>,<NA>,<NA>,2014-12-10 15:16:16.259000+00:00,2014-12-20 21:36:42.139000+00:00,https://swapi.dev/api/species/2/


#### 1.2.7 Clean starships & vehicles dataframes

In [35]:
# Convert cost_in_credits (string) into int 

# Replace 'unknown' with NaN
replace_string = ['unknown']
starships['cost_in_credits'] = starships['cost_in_credits'].replace(replace_string, pd.NA)
vehicles['cost_in_credits'] = vehicles['cost_in_credits'].replace(replace_string, pd.NA)

# Convert to int or float
starships['cost_in_credits'] = starships['cost_in_credits'].astype('Int64')
vehicles['cost_in_credits'] = vehicles['cost_in_credits'].astype('Int64')

# Double check
print(starships['cost_in_credits'].dtypes)
print(vehicles['cost_in_credits'].dtypes)

Int64
Int64


In [36]:
# Select needed columns and store dataframes

# Starships
selected_columns = ['id', 'name', 'model', 'cost_in_credits', 
                    'starship_class', 'created', 'edited', 'url']
starships_cleaned = starships[selected_columns]
dataframes_cleaned['starships'] = starships_cleaned

# Vehicles
selected_columns = ['id', 'name', 'model', 'cost_in_credits', 
                    'vehicle_class', 'created', 'edited', 'url']
vehicles_cleaned = vehicles[selected_columns]
dataframes_cleaned['vehicles'] = vehicles_cleaned

In [37]:
# final dataframe for starships
dataframes_cleaned['starships'].head(2)

,id,name,model,cost_in_credits,starship_class,created,edited,url
0,2,CR90 corvette,CR90 corvette,3500000,corvette,2014-12-10 14:20:33.369000+00:00,2014-12-20 21:23:49.867000+00:00,https://swapi.dev/api/starships/2/
1,3,Star Destroyer,Imperial I-class Star Destroyer,150000000,Star Destroyer,2014-12-10 15:08:19.848000+00:00,2014-12-20 21:23:49.870000+00:00,https://swapi.dev/api/starships/3/


In [38]:
# final dataframe for vehicles
dataframes_cleaned['vehicles'].head(2)

,id,name,model,cost_in_credits,vehicle_class,created,edited,url
0,4,Sand Crawler,Digger Crawler,150000,wheeled,2014-12-10 15:36:25.724000+00:00,2014-12-20 21:30:21.661000+00:00,https://swapi.dev/api/vehicles/4/
1,6,T-16 skyhopper,T-16 skyhopper,14500,repulsorcraft,2014-12-10 16:01:52.434000+00:00,2014-12-20 21:30:21.665000+00:00,https://swapi.dev/api/vehicles/6/


### 1.3 Load

In [39]:
# Initiatilise environment
conn = sqlite3.connect('starwars.db')
c = conn.cursor()

In [40]:
# Load primary datasets
for key, df in dataframes_cleaned.items():
    print(f'SQL Loading Dataset: {key}')
    c.execute(f"CREATE TABLE IF NOT EXISTS {key} ({', '.join(df.columns)})")
    conn.commit()
    df.to_sql(f'{key}', conn, if_exists='replace', index = False)

SQL Loading Dataset: films
SQL Loading Dataset: people
SQL Loading Dataset: planets
SQL Loading Dataset: species
SQL Loading Dataset: starships
SQL Loading Dataset: vehicles


In [41]:
# Load junction tables
for key, df in junction_tables.items():
    print(f'SQL Loading Dataset: {key}')
    c.execute(f"CREATE TABLE IF NOT EXISTS {key} ({', '.join(df.columns)})")
    conn.commit()
    df.to_sql(f'{key}', conn, if_exists='replace', index = False)

SQL Loading Dataset: character_in_film
SQL Loading Dataset: planet_in_film
SQL Loading Dataset: starship_in_film
SQL Loading Dataset: species_in_film
SQL Loading Dataset: vehicle_in_film


## 2. The Database Schema

I've created a schema of these databases. See below:


<img src="Pictures/schema.png" width="1000" />

## 3. Analysis

In [42]:
con = sqlite3.connect('starwars.db')

### 3.1 INNER JOIN: compare the overall average height of that species to the average height of characters of that species

Logic (see graph below):
1. Inner Join 'people' table and 'species' table using 'people.species = species.id'
    - Inner join automatically remove 
        - characters who do not have a species
        - species that do not have a character
2. GROUP results BY species name to get aggregate calculations for each species
    - Use AVG() to count the average height of characters within each species
3. Use WHERE to filter out species that don't have an average height recorded
4. ORDER the result first by the average overall height of species, second by average character height of each species

<img src="Pictures/inner_join.png" width="450" />

In [43]:
query = """
SELECT
    s.id AS species_id,
    s.name AS species_name,
    s.average_height AS avg_species_height,
    ROUND(AVG(p.height), 1) AS avg_character_height
FROM
    people AS p
INNER JOIN 
    species AS s
ON 
    p.species = s.id
WHERE
    s.average_height IS NOT NULL
GROUP BY
    s.name
ORDER BY
    avg_species_height DESC, avg_character_height DESC;
"""

pd.read_sql_query(sql=query, con=con)

,species_id,species_name,avg_species_height,avg_character_height
0,5,Hutt,300,175.0
1,25,Quermian,240,264.0
2,32,Kaminoan,220,221.0
3,3,Wookie,210,231.0
4,20,Cerean,200,198.0
5,7,Trandoshan,200,190.0
6,15,Twi'lek,200,179.0
7,19,Toong,200,163.0
8,12,Gungan,190,208.7
9,37,Pau'an,190,206.0


### 3.2 LEFT JOIN: Are the number of characters living on a planet and the number of films featuring that planet correlated?

Logic (see graph below):
1. LEFT JOIN films, planet_in_film, planets, people tables, by following matching keys
    - films.id = planet_in_film.film_id
    - planet_in_film.planet_id = planets.id
    - planets.id = people.residence_planet
2. GROUP BY planet ID to get aggregated counts per planet
3. SELECT relevant columns & COUNT:
    - COUNT the number of character living on a planet, by counting DISTINCT people_id
    - COUNT the number of films featuring a planet, by counting DISTINCT film_id
4. ORDER the results BY 
    - the number of characters living on a planet
    - the number of films featuring that planet

<img src="Pictures/left_join.png" width="700" />

In [44]:
query = """
SELECT
    planets.id AS planet_id,
    planets.name AS planet_name,
    COUNT(DISTINCT people.id) AS n_characters,
    COUNT(DISTINCT films.id) AS n_films

-- Join 4 tables
FROM 
    films
LEFT JOIN 
    planet_in_film AS pif
ON 
    films.id = pif.film_id
LEFT JOIN
    planets
ON 
    pif.planet_id = planets.id
LEFT JOIN
    people
ON
    planets.id = people.residence_planet

-- Grouping by planet ID to get aggregated counts per planet
GROUP BY
    planets.id

-- Order the result by number of characters and number of films
ORDER BY
    n_characters DESC, n_films DESC;
"""

pd.read_sql_query(sql=query, con=con)

,planet_id,planet_name,n_characters,n_films
0,8,Naboo,11,4
1,1,Tatooine,10,5
2,9,Coruscant,3,4
3,2,Alderaan,3,2
4,10,Kamino,3,1
5,14,Kashyyyk,2,1
6,6,Bespin,1,1
7,7,Endor,1,1
8,11,Geonosis,1,1
9,12,Utapau,1,1


In [45]:
# Calculate the correlation between the number of characters living on a planet and the number of films featuring that planet 
df = pd.read_sql_query(sql=query, con=con)
df['n_characters'].corr(df['n_films'])

0.8030726477262778

### 3.3 GROUP BY: Total Cost of Starships Featured in Each Film, Ordered by Total Cost

To obtain the data (see graph below):
1. We first use the junction table 'starship_in_film' to JOIN the 'films' table and 'starships' table. 
2. Then, we GROUP the new table BY film id.
3. Next,
    - COUNT the number of starships featured in each film.
    - SUM the 'cost_in_credits' of starships featured in each film.
4. Last, we order the table by the 'total_starship_cost'.

<img src="Pictures/group_by.png" width="700" />

In [46]:
query = """
SELECT
    f.title AS film_title,
    f.episode_id AS episode_id,
    strftime('%Y', f.release_date) AS release_date,
    COUNT(DISTINCT s.id) AS n_starships, 
    SUM(s.cost_in_credits) AS total_cost_of_starships
FROM 
    films AS f
LEFT JOIN 
    starship_in_film as sif
ON 
    f.id = sif.film_id
LEFT JOIN
    starships as s
ON
    sif.starship_id = s.id
GROUP BY
    f.id
ORDER BY
    total_cost_of_starships DESC
"""

pd.read_sql_query(sql=query, con=con)

,film_title,episode_id,release_date,n_starships,total_cost_of_starships
0,A New Hope,4,1977,8,1000154124998
1,Return of the Jedi,6,1983,12,1410369998
2,The Empire Strikes Back,5,1980,9,1302474998
3,Revenge of the Sith,3,2005,12,246425500
4,The Phantom Menace,1,1999,5,55200000
5,Attack of the Clones,2,2002,9,2415700


### 3.4 HAVING: Iconic Starships in Star Wars films

Definition of iconic starships:
- Starships that show up in at least three star wars films

Logic (see graph below):
1. LEFT JOIN films, starships, starship_in_film tables
2. GROUP BY starship_id, to calculate aggregate statistics
    - COUNT DISTINCT film_id to get the number of films in which each starship appeared.
3. The Having clause is executed last to filter out starships that appeared in fewer than 3 films.

<img src="Pictures/having.png" width="500" />

In [47]:
query = """
SELECT
    s.id AS starship_id,
    s.name AS starship_name,
    COUNT(DISTINCT f.id) AS n_films
FROM 
    starships AS s
LEFT JOIN 
    starship_in_film as sif
ON 
    s.id = sif.starship_id
LEFT JOIN
    films as f
ON
    sif.film_id = f.id
GROUP BY
    s.id

-- Filter results to only include starships that appeared in 3 or more films
HAVING 
    n_films >= 3

-- Order the results by the number of film appearances (in descending order) 
-- and then alphabetically by starship name
ORDER BY
    n_films DESC, starship_name ASC;
"""

pd.read_sql_query(sql=query, con=con)

,starship_id,starship_name,n_films
0,2,CR90 corvette,3
1,32,Droid control ship,3
2,10,Millennium Falcon,3
3,3,Star Destroyer,3
4,12,X-wing,3
5,11,Y-wing,3


### 3.5 COMMON TABLE EXPRESSION (CTE): Iconic Characters of Star Wars Species

Definition of Iconic Characters of Star Wars Species:
- The top 3 characters for each species, based on the number of films that they have appeared in.
- These characters must appear in at least 2 films.
- Character with unknown species are grouped together and treated as one unique species for ranking purposes.


SQL Process (see graph below):
1. Join -> character_apperances (CTE1)
    1. Use the junction table 'character_in_film' to join 'films' table and 'people' table.
    2. Join the resulting table with the 'species' table using the 'species' column in 'people' and the 'id' column in 'species'
    3. Select needed columns
    4. Count the number of films that each character appeared in by using COUNT(DISTINCT f.id)
2. character_apperances (CTE1) -> ranked_characters (CTE2)
    - Use ROW_NUMBER() to rank characters within each species according to the number of films that each character appeared in
3. ranked_characters (CTE2) -> final table
    - Use WHERE to choose characters who 
        - appeared in at least 2 films and 
        - are top 3 for each species

<img src="Pictures/CTE.png" width="850" />

In [48]:
query = """
-- CTE_1: Count the number of films for each character.
WITH character_apperances AS (
    SELECT
        p.id AS character_id,
        p.name AS character_name,
        s.name AS species_name, 
        -- Count the number of films that each character appeared in
        COUNT(DISTINCT f.id) AS n_films
    FROM 
        films AS f
    LEFT JOIN
        character_in_film AS cif
    ON 
        f.id = cif.film_id
    LEFT JOIN
        people AS p
    ON 
        cif.character_id = p.id
    LEFT JOIN 
        species AS s
    ON
        p.species = s.id
    GROUP BY
        p.id
),

-- CTE_2: rank characters based on their film appearances within each species.
ranked_characters AS (
    SELECT
        character_id,
        character_name,
        species_name,
        n_films,
        ROW_NUMBER() OVER (PARTITION BY species_name ORDER BY n_films DESC) AS rank
    FROM 
        character_apperances
)

-- Main Query: Filter characters who've appeared in 2 or more films 
-- and are ranked in the top 3 within each species.
SELECT
    *
FROM 
    ranked_characters
WHERE 
    n_films >= 2 AND rank < 4
ORDER BY 
    species_name ASC, rank ASC; 
"""

pd.read_sql_query(sql=query, con=con)

,character_id,character_name,species_name,n_films,rank
0,10,Obi-Wan Kenobi,None,6,1
1,21,Palpatine,None,5,2
2,1,Luke Skywalker,None,4,3
3,52,Ki-Adi-Mundi,Cerean,3,1
4,59,Mas Amedda,Chagrian,2,1
5,2,C-3PO,Droid,6,1
6,3,R2-D2,Droid,6,2
7,63,Poggle the Lesser,Geonosian,2,1
8,36,Jar Jar Binks,Gungan,2,1
9,67,Dooku,Human,2,1


### 3.6 UNION: average hours needed to finish an entry

My logics (see graph below):
- UNION ALL 
    - UNION ALL table entries with their created time and last edited time
    - Choose UNION ALL instead UNION to avoid deleting rows with same values across tables
    - Use python .join method to construct the query of unioning all tables
- Common Table Expression
    - Use Common Table Expression to refer to the unioned table
- Calculate time differences
    - Calculate time difference in seconds using strftime()
    - Average the time difference in seconds
    - Divide average time by 3600 to get time difference in hours
    - Round it up with 2 decimals

<img src="Pictures/union.png" width="500" />

In [49]:
# List of tables to union
tables = ["films", "planets", "species", "people", "starships", "vehicles"]

# Construct UNION ALL parts based on the tables list
union_all_query = "\nUNION ALL\n".join(
    f"SELECT created, edited FROM {table}" for table in tables
)

# Construct the final query
# 1. Use a WITH clause to create all_entries table with all the 'created' and 'edited' timestamps from all tables
# 2. Calculate the average difference between 'edited' and 'created' timestamps in hours
query = f"""
WITH all_entries AS (
    {union_all_query}
)

SELECT 
    ROUND(
        AVG(strftime('%s', edited) - strftime('%s', created)) / 3600, 2
        ) AS avg_diff_hours
FROM 
    all_entries
"""

# Read the SQL query using pandas
pd.read_sql_query(sql=query, con=con)

,avg_diff_hours
0,79.89
